## - - 이전 코드 시작 - -

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 2.8 MB 44.9 MB/s 
     |████████████████████████████████| 806 kB 42.1 MB/s 
     |████████████████████████████████| 829 kB 32.7 MB/s 
     |████████████████████████████████| 136 kB 46.0 MB/s 
     |████████████████████████████████| 398 kB 40.9 MB/s 
     |████████████████████████████████| 636 kB 42.2 MB/s 
     |████████████████████████████████| 3.3 MB 40.3 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 895 kB 13.2 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
     |████████████████████████████████| 96 kB 5.7 MB/s 
     |████████████████████████████████| 144 kB 43.1 MB/s 
     |████████████████████████████████| 271 kB 46.7 MB/s 
     |████████████████████████████████| 94 kB 368 kB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [5]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)


In [6]:
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:00, 71.4MB/s]
Downloading: 3.88MB [00:00, 66.0MB/s]                  


In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [8]:
#토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [9]:
#학습 데이터셋 구축
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from train dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 57688/57688 [07:59<00:00, 120.19it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 ##향 ##곡 ##을 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 [SEP]
INFO:ratsnlp:answer: 교 ##향 ##곡
INFO:ratsnlp:features: QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966

In [10]:
train_dataset[0]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## - - 이전 코드 끝 - -

QAFeatures(  
  input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3],  
  attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],    
  token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
  start_positions : 45   
  end_positions : 47  
)

이전 코드에서 설정해 두었던 `doc_stride`라는 인자에 주목할 필요가 있습니다.  
질의응답 태스크를 수행할 때 지문의 길이는 BERT 모델이 처리할 수 있는 `max_seq_length`보다 긴 경우가 많습니다. 이럴 때는 어쩔 수 없이 질문은 그대로 두고 지문 일부를 편집하는 방식으로 학습 데이터를 만들 수밖에 없습니다.

예를 들어 `doc_stride`를 64로 설정해 둔다면 원래 지문 앞부분 64개 토큰(`18 ##3 ##9년 … 좌 ##절 ##과 실망 ##에`)을 없애고 원래 지문 뒷부분에 이어지고 있던 64개 토큰(`##벤 ##의 … 정신적 피로 ##나`)을 가져와 붙입니다.   
다음과 같습니다.

**[CLS]** 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? **[SEP]** <u>18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 ##향 ##곡 ##을 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에</u> 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 **[SEP]**  

cf) 밑줄 친 곳은 다음 인스턴스 만들 때 삭제 대상입니다.

그런데 `max_seq_length` 때문에 지문을 이처럼 인위적으로 편집하면 정답 토큰 시퀀스(`교 ##향 ##곡`)도 함께 날아갈 수 있습니다. 이 경우 `start_positions`, `end_positions` 모두 `0`을 줍니다. 실제로 다음에 해당하는 전처리 최종 결과는 `train_dataset[1]`인데요, `train_dataset[1].start_positions`, `train_dataset[1].end_positions`을 확인해 보면 모두 `0`임을 알 수 있습니다.

**[CLS]** 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? **[SEP]** 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 <u>파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 ##벤 ##의 교 ##향 ##곡 9 ##번 ##을 듣고 깊은 감 ##명을 받았는데 , 이것이 이 ##듬 ##해 1월 ##에 파 ##우스 ##트 ##의 서 ##곡 ##으로 쓰여 ##진 이</u> **[SEP]**

cf) 밑줄 친 곳은 이번 인스턴스 만들 때 이어 붙인 토큰입니다.

In [11]:
#학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

평가용 데이터 로더를 구축하려면 다음 코드를 실행합니다.  
평가용 데이터셋(`val_dataset`) 구축 방식은 학습용(`train_dataset`)과 동일합니다. 데이터 로더와 관련한 자세한 내용은 <4-2>를 다시 복습하세요.

In [12]:
#평가용 데이터 구축
from torch.utils.data import SequentialSampler
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 5533/5533 [01:13<00:00, 74.78it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##위반 ) 으로 지명 ##수 ##배 ##되었다 . 198 ##9년 3월 12 ##일 서울 ##지 ##방 ##검찰 ##청 공안 ##부는 임종석 ##의 사전 ##구속 ##영장 ##을 발부 ##받았 ##다 . 같은 해 6월 30 ##일 평양 ##축 ##전에 임 ##수 ##경을 대표로 파견 ##하여 국가보안법 ##위반 혐의가 추가 ##되었다 . 경찰은 12월 18 ##일 ~ 20 ##일 사이 서울 경 ##희 ##대학 ##교 ##에서 임종석 ##이 성 ##명 발표 ##를 추진 ##하고 있다는 첩 ##보를 입 ##수 ##했고 , 12월 18 ##일 오전 7시 40 ##분 경 가스 ##총 ##과 [SEP]
INFO:ratsnlp:answer: 임 ##수 ##경을
INFO:ratsnlp:features: QAFeatures(input_ids=[2, 25704, 29697, 23343, 8476, 4046, 14601, 4728, 8409, 24345, 20158, 902, 10393, 4057, 32, 3, 25704, 29697, 18163, 26757, 20604, 29804, 10901, 25099, 13225, 4047, 16704, 11, 10901, 12175, 4086, 11417, 4113, 4337,

<**6단계**> **모델 불러오기**  

다음 코드를 수행해 모델을 초기화합니다. `BertForQuestionAnswering`은 프리트레인을 마친 BERT 모델 위에 <7-1>에서 설명한 질의응답용 태스크 모듈이 덧붙여진 형태의 모델 클래스입니다.

In [13]:
#모델 초기화
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

<**7단계**> **모델 학습시키기**   

파이토치 라이트닝이 제공하는 `lightning` 모듈을 상속받아 모델과 옵티마이저, 학습 과정 등이 정의된 태스크를 정의합니다. 다음 코드를 실행하면 질의응답용 태스크(task)를 정의할 수 있습니다.  
모듈은 방금 코드에서 준비한 모델 클래스를 사용하고, 옵티마이저로는 아담(Adam), 러닝 레이트 스케줄러로는 `ExponentialLR`을 사용합니다.

In [14]:
#태스크 정의
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

다음 코드를 실행하면 트레이너를 정의할 수 있습니다.   
이전 챕터에도 나왔었지만 이 트레이너는 GPU/TPU 설정, 로그와 체크포인트 등 귀찮은 설정들을 알아서 해줍니다.

In [15]:
#트레이너 정의
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


마지막으로 트레이너의 `fit()` 함수를 호출하면 학습을 시작합니다.

In [16]:
#학습 개시  
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


자금까지 우리는 질의응답 모델을 학습하는 과정을 실습했습니다.  
질의응답 모델은 입력 토큰 각각에 대해 어떤 범주에 속할 확률을 반환한다는 점에서 시퀀스 레이블링(6장)과 비슷하지만 입출력이 살짝 다릅니다. 그 내용을 정리하면 다음과 같습니다.


|방법론|입력|출력|대표 과제|
|:---:|:---:|:---:|:---:|
|질의응답|문서 2개(질문+지문)|토큰 각각이 정답의 시작/끝일 확률|KorQuAD|
|시퀀스 레이블링|문서 1개|토큰 각각의 범주 확률|개체명 인식|      
  

   
<center><표 - 시퀀스 레이블링과 질의응답 모델 비교></center>    
